script completo para realizar clasificación supervisada usando datos de polarización VV y VH de Sentinel-1, empleando Google Earth Engine (GEE) y Random Forest. Este ejemplo es ideal para diferenciar tipos de cobertura vegetal como Mombasa, suelo y árboles.

In [ ]:
import ee
import  geemap
from IPython.display import display
ee.Authenticate()
ee.Initialize(project='my-project-unimagdalena')

# --- 1. Definir zona de estudio y fechas ---
aoi = ee.Geometry.Rectangle([-76.0, 8.6, -75.7, 8.9])  # Ajusta según tu región
start = '2025-01-01'
end = '2025-06-01'

# --- 2. Cargar colección Sentinel-1 y calcular media de VV y VH ---
s1 = (ee.ImageCollection('COPERNICUS/S1_GRD')
      .filterBounds(aoi)
      .filterDate(start, end)
      .filter(ee.Filter.eq('instrumentMode', 'IW'))
      .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
      .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
      .select(['VV', 'VH'])
      .mean()
      .clip(aoi))

# --- 3. Crear puntos de entrenamiento manuales con clases ---
# Clases: 1 = Mombasa, 2 = Suelo, 3 = Árboles
mombasa = ee.Feature(ee.Geometry.Point([-75.85, 8.75]), {'class': 1})
suelo   = ee.Feature(ee.Geometry.Point([-75.87, 8.74]), {'class': 2})
arboles = ee.Feature(ee.Geometry.Point([-75.86, 8.76]), {'class': 3})
training_points = ee.FeatureCollection([mombasa, suelo, arboles])

# --- 4. Extraer valores VV y VH para entrenamiento ---
training = s1.sampleRegions(
    collection=training_points,
    properties=['class'],
    scale=10
)

# --- 5. Entrenar clasificador Random Forest ---
classifier = ee.Classifier.smileRandomForest(50).train(
    features=training,
    classProperty='class',
    inputProperties=['VV', 'VH']
)

# --- 6. Clasificar la imagen ---
classified = s1.classify(classifier)

# --- 7. Visualizar resultados ---
Map = geemap.Map(center=[8.75, -75.88], zoom=12)
Map.addLayer(s1.select('VV'), {'min': -20, 'max': 0}, 'VV')
Map.addLayer(s1.select('VH'), {'min': -25, 'max': -5}, 'VH')
Map.addLayer(classified, {
    'min': 1, 'max': 3,
    'palette': ['green', 'brown', 'darkgreen']
}, 'Clasificación supervisada')

# --- 8. Exportar imagen clasificada a Google Drive ---
export = ee.batch.Export.image.toDrive(
    image=classified,
    description='Clasificacion_Sentinel1',
    folder='GEE_Exports',
    fileNamePrefix='clasificacion_s1',
    region=aoi,
    scale=10,
    crs='EPSG:4326'
)
export.start()
print("Exportación iniciada a Google Drive...")

# Define the colors using RGB tuples for the legend
legend_colors = [(0, 128, 0),  # Green for Mombasa
                 (139, 69, 19), # Brown for Suelo
                 (0, 100, 0)]  # Darkgreen for Árboles

# Add the legend with the correct color format
Map.add_legend(title='Clases de cobertura', labels=['Mombasa', 'Suelo', 'Árboles'], colors=legend_colors)

Map
